<a href="https://colab.research.google.com/github/aharrisonau/gym-ReverseTruck/blob/main/ReverseTruck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ReverseTruck uses the ReverseTruck_env to simulate the reversing of a truck and trailer combination into a loading bay

The environment and the simulation have been developed by Andrew Harrison as a submission for SIT796 - Reinforcement Learning at Deakin University

First step is to install the environment for Github

In [1]:
# helper code from https://stackoverflow.com/questions/55835318/how-to-create-and-use-a-custom-openai-gym-environment-on-google-colab
%%capture
!rm -r ReverseTruck_env
!git clone https://github.com/aharrisonau/gym-ReverseTruck.git
!pip install -e gym-ReverseTruck




In [2]:
import gym_ReverseTruck

In [3]:
# preliminaries
import pandas as pd
import numpy as np
import math as math
import gym
from gym import error, spaces, utils
from gym.utils import seeding

Then we can create the environment

Note that the environment requires a truck starting position, a truck setup file and a dictionary of obstacles passed to it

In [4]:

# the initial truck model has a prime mover length of 5m and a trailer of 10m
# width of both is set as 2.5m, but not used yet
TruckDefinition = np.array( [5.0, 2.5, 10.0, 2.5])

# The truck starts with the back of the truck 20m out and 20m forward of the 
# end point (0,0) and perpendicular to the finish position
# Note that the position is consistent with the truck definition
StartPosition = np.array( [[20.0,35.0],
                         [20.0,30],
                         [20.0,20.0]])

# No obstacles are used yet
Obstacles = {}


In [5]:
env = gym.make('ReverseTruck-v0',TruckDefinition=TruckDefinition, StartPosition=StartPosition, Obstacles=Obstacles)

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Follow the method of Practical 1 to run the model

We need to define a policy to take actions.

The simplest policy is to make the back of the truck follow a 20m radius curve till it gets to the loading dock (x=0)

The geometry of this means that the trailer angle should be held at 0.253 rad to the direction of the prime mover.

The policy is then if the trailer angle is less than this, apply "alpha" right steering to increase the angle
If angle greater than this, apply "alpha" left steering to decrease the angle.

"alpha" is an adjustable parameter.  1 means full 45 deg steerign lock.  Lower it to prevent overcorrection if needed

In [21]:
def policy(obs, t):
    # Write the code for your policy here. You can use the observation
    # (a tuple of position and velocity), the current time step, or both,
    # if you want.
    pmX, pmY, pivX, pivY, trlX, trlY = obs
    alpha=0.2

    pmVect=np.array([pmX-pivX,pmY-pivY]) # prime mover vector
    trlVect=np.array([pivX-trlX,pivY-trlY]) # trailer vector
    pm_trlSin = np.cross(trlVect,pmVect,)/np.linalg.norm(pmVect)/np.linalg.norm(trlVect) #sine of the trailer angle relative to the PM
    pm_trlAngle=math.asin(np.clip(pm_trlSin,-1,1)) #relative trailer angle.  Clip eliminate rounding errors giving arguments >1    

    if pm_trlAngle<0.253:
      actions=[-1.0,-alpha]
    else:
      actions=[-1.0,alpha]
    
   
    return actions

In [22]:
o = env.reset()

for t in range(350):
    
    action = policy(o,t)  # Call your policy
    o, r, d, _ = env.step(action)  # Pass the action chosen by the policy to the environment
    print(o,action)

    if d and t<TIME_LIMIT-1:
        print("Task completed in", t, "time steps")
        break
else:
    print("Time limit exceeded. Try again.")

env.reset()

[19.98416156 34.89997491 20.         29.9        20.         19.9       ] [-1.0, -0.2]
[19.96864004 34.79990016 20.00031677 29.8000005  20.         19.80000051] [-1.0, -0.2]
[19.95343561 34.69977674 20.0009503  29.70000251 19.99999683 19.70000255] [-1.0, -0.2]
[19.93854841 34.59960566 20.0019006  29.60000702 19.9999873  19.60000721] [-1.0, -0.2]
[19.9239786  34.49938792 20.00316764 29.50001505 19.99996817 19.50001556] [-1.0, -0.2]
[19.90972632 34.39912454 20.00475142 29.40002759 19.99993618 19.40002875] [-1.0, -0.2]
[19.89579172 34.29881651 20.00665192 29.30004566 19.99988803 19.30004794] [-1.0, -0.2]
[19.88217493 34.19846484 20.00886913 29.20007024 19.99982041 19.20007433] [-1.0, -0.2]
[19.86887609 34.09807054 20.01140301 29.10010235 19.99972996 19.10010916] [-1.0, -0.2]
[19.85589533 33.99763462 20.01425355 29.00014298 19.99961328 19.0001537 ] [-1.0, -0.2]
[19.84323279 33.89715808 20.01742072 28.90019315 19.99946696 18.90020927] [-1.0, -0.2]
[19.8308886  33.79664194 20.02090447 28.800

array([20., 35., 20., 30., 20., 20.])